In [34]:
import math
import numpy as np
import h5py
from tqdm import tqdm
import scipy as sp
from scipy.sparse import csr_matrix

In [35]:
hf = h5py.File('clean_run_0102.h5', 'r')

In [36]:
events = hf['clean']

In [37]:
print(len(events))

69631


In [38]:
DETECTOR_LENGTH = 1000.0
DETECTOR_RADIUS = 275.0

x_disc = 20
y_disc = 20
z_disc = 20

x_inc = (2*DETECTOR_RADIUS)/x_disc
y_inc = (2*DETECTOR_RADIUS)/y_disc
z_inc = DETECTOR_LENGTH/z_disc

In [39]:
discElements = x_disc*y_disc*z_disc

buckets = []
charges = []
downsampled = []

test_events = list(range(100))

for i in tqdm(test_events):
    evt = str(i)
    num_pts = 0
    discretized_xyz = np.zeros([events[evt].shape[0],events[evt].shape[1]])
    for point in events[evt]:
        x_bucket = math.floor(((point[0]+DETECTOR_RADIUS)/(2*DETECTOR_RADIUS))*x_disc)
        y_bucket = math.floor(((point[1]+DETECTOR_RADIUS)/(2*DETECTOR_RADIUS))*y_disc)
        z_bucket = math.floor((point[2]/DETECTOR_LENGTH)*z_disc)

        bucket_num = z_bucket*x_disc*y_disc + x_bucket + y_bucket*x_disc
    
        #sparse matrix implementation
        buckets.append(bucket_num)
        charges.append(point[3]/10)
    
        avg_x = -DETECTOR_RADIUS + ((2*x_bucket+1)*x_inc)/2.0
        avg_y = -DETECTOR_RADIUS + ((2*y_bucket+1)*y_inc)/2.0
        avg_z = ((2*z_bucket+1)*z_inc)/2.0                          
    
        discretized_xyz[num_pts] = [avg_x, avg_y, avg_z, point[3], point[4], point[5], point[6]]
    
        num_pts += 1

    #create csr matrix where data is chrage and row np.zeros(len(col)) of size (1, discElements)
    cols = buckets
    rows = np.zeros(len(cols))
    data = charges

    #automatically sums data entries for data occuring at the same point
    #no need for sum_duplicates()
    discretized_data_sparse_CHARGE  = sp.sparse.csr_matrix((data, (rows, cols)), shape=(1, discElements))
    
    downsampled.append(discretized_xyz)

100%|██████████| 100/100 [00:02<00:00, 43.54it/s]


In [40]:
print(len(downsampled))

100


In [41]:
downsampled[0][2]

array([  13.75      ,  -68.75      ,  175.        , 1520.95757226,
       1065.        ,    8.        ,  209.78130223])

In [43]:
f = h5py.File("downsampled-test-small.h5", "w")

In [45]:
dset = f.create_dataset("features", data=downsampled)

TypeError: Object dtype dtype('O') has no native HDF5 equivalent

In [ ]:
f.close()